In [ ]:
# Install necessay libraries
!pip install qiskit
!pip install qiskit_ibm_runtime
!pip install pylatexenc
!pip install matplotlib

In [3]:
# Import necessary libraries
# Built-in modules
import math

# Imports from Qiskit
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit import transpile, QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.providers.basic_provider import BasicProvider

In [4]:
# Use "basic_simulator" as backend provider to simulate quantum circuit
backend = BasicProvider().get_backend('basic_simulator')

In [5]:
# Define a Grover oracle which marks the states corresponding to the desired even numbers
# During marking we will consider the 0th qubit always in state 0 because
# this is the signature of a even number like: 010, 1100, 1110, 101010
def grover_oracle(marked_states, num_qubits):
    """Build a Grover oracle for multiple marked states

    Parameters:
        marked_states (list): Marked states of oracle
        num_qubits (int): Number of qubits

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    qc = QuantumCircuit(num_qubits)

    # We keep track of the marked even numbers to avoid marking them twice
    done_mark = []

    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        if rev_target[1:] not in done_mark:
            # Find the indices of all the '0' elements in bit-string
            zero_inds = [0] # 0th qubit is always 0 for even numbers
            zero_inds.extend([ind for ind in range(1, num_qubits) if rev_target.startswith("0", ind)])
            # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
            # where the target bit-string has a '0' entry
            qc.x(zero_inds)
            qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
            qc.x(zero_inds)
            done_mark.append(rev_target[1:])
    return qc

In [6]:
def odd_to_even(n, lst):
    """
    Build a function that uses quantum circuit to convert
    all odd numbers to even numbers

    Parameters:
        n (int): The maximum number
        lst (list): List of numbers

    Returns:
        list: A list where all numbers are even
    """
    # Calculate the number of required qubits
    k = math.ceil(math.log2(n))

    # Convert the list of numbers from decimal to binary
    marked_states = [bin(num)[2:].zfill(k) for num in lst]

    # Prepare an oracle for the marked states
    oracle = grover_oracle(marked_states, k)

    # Count the number of multi-control z gate in the oracle.
    # This represents the number of unique marked elements
    num_marked = dict(oracle.count_ops())["ccx" if k==3 else f"c{k-1}z"]

    # Append the diffusion operator
    grover_op = GroverOperator(oracle)

    # Determine the optimal number of iterations
    optimal_num_iterations = max(1, math.floor(
        math.pi / (4 * math.asin(math.sqrt(5 / 2**grover_op.num_qubits)))
        ))

    # Everything is set. Prepare the Quantum Circuit
    qc = QuantumCircuit(k)

    # Create even superposition of all basis states
    qc.h(range(grover_op.num_qubits))

    # Apply Grover operator the optimal number of times
    qc.compose(grover_op.power(optimal_num_iterations), inplace=True)

    # Measure all qubits
    qc.measure_all()

    # Draw the final circuit
    qc.decompose().draw(output="mpl", style="iqp")

    # Simulate the circuit
    tqc = transpile(qc, backend)
    counts = backend.run(tqc, shots=10000).result().get_counts()

    # Selected even numbers
    marked_even = sorted(counts, key=counts.get, reverse=True)[:num_marked]

    # Convert to decimal
    marked_even = [int(j, 2) for j in marked_even]
    # Delete 0 from the list since we have to remain within [1,n)
    if 0 in marked_even:
        marked_even.remove(0)

    # Prepare the even list
    ev_list = []
    for j in lst:
        if j in marked_even:
            ev_list.append(j)
        elif j+1 in marked_even:
            ev_list.append(j+1)
        else:
            ev_list.append(j-1)

    return ev_list

In [8]:
n = 31
lst = [1,2,2,4,5,6,7,11,25,26,28,29]
ev_lst = odd_to_even(n, lst)
print(ev_lst)

[2, 2, 2, 4, 6, 6, 6, 10, 26, 26, 28, 28]
